In [103]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [104]:
#Parameters
samplenum = 5
batchsize = samplenum
epochs = 500
time_length = 60; #seconds


In [122]:
# Generate simulation
sim = d.PySimSeq('test2.sim', time_length)
ySeq = sim.compute(sim.p)
f = sim.f(sim.y, sim.ydot, sim.yddot, sim.p)
df = sim.df_dp(sim.y, sim.ydot, sim.yddot, sim.p)
ySeq.shape

(180,)

In [106]:
#Sample targets only variables in z direction
x = np.zeros((samplenum,3))
x[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
x[:,1] = 2

x

array([[0.        , 2.        , 0.79004199],
       [0.        , 2.        , 0.18758779],
       [0.        , 2.        , 0.62876515],
       [0.        , 2.        , 0.78892204],
       [0.        , 2.        , 0.62024301]])

In [107]:
#Sample ptraj
start_time1 = time.time()
y = np.zeros((samplenum, 3*time_length))
for i in range(samplenum):
    y[i, :] = sim.sample_ptraj(x[i, :], sim.p)

print(f'\nDuration: {(time.time() - start_time1)/60:.3f} min') # print the time elapsed


Duration: 0.149 min


In [108]:
#Preprocesing
x= torch.tensor(x)
y= torch.tensor(y)

In [109]:
print(x.shape)
print(y.shape)

torch.Size([5, 3])
torch.Size([5, 180])


In [115]:
#build model class
class Dataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

dataset = Dataset(x,y)

train_loader = DataLoader(dataset = dataset, batch_size = batchsize, shuffle = True)

class Model(nn.Module):

    def __init__(self, n_in, out_sz, hlayers):
        super().__init__()

        self.bnorm = nn.BatchNorm1d(n_in)

        layerlist = []

        for i in hlayers:
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            n_in = i
        layerlist.append(nn.Linear(hlayers[-1],out_sz))

        self.layers = nn.Sequential(*layerlist)

    def forward(self, x):

        x = self.bnorm(x)
        x = self.layers(x)
        return x

In [116]:
#Define Model
model = Model(x.shape[1], y.shape[1], [200,100])

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(model.layers)


Sequential(
  (0): Linear(in_features=3, out_features=200, bias=True)
  (1): ReLU(inplace=True)
  (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=200, out_features=100, bias=True)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=100, out_features=180, bias=True)
)


In [112]:
#Split into Test/Train data
#batch_size = 2
#test_size = int(batch_size * .2)

# cat_train = cats[:batch_size-test_size]
# cat_test = cats[batch_size-test_size:batch_size]
# con_train = conts[:batch_size-test_size]
# con_test = conts[batch_size-test_size:batch_size]
#y_train = y[:batch_size-test_size]
#y_test = y[batch_size-test_size:batch_size]
x_train = x.float()
y_train = y.float()

In [113]:
#Train the model
start_time = time.time()

losses = []

for i in range(epochs):
    for inputs,labels in train_loader:
        x_train = inputs.float()
        y_train = labels.float()
        i+=1
        y_pred = model(x_train)

        loss = torch.sqrt(criterion(y_pred, y_train)) # RMSE
        losses.append(loss)

        if i%25 == 1:
            print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')

        #Clear the gradient buffer (w <-- w - lr*gradient)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

epoch:   1/500  loss: 1.82144845
epoch:  26/500  loss: 0.39382920
epoch:  51/500  loss: 0.11501388
epoch:  76/500  loss: 0.05033321
epoch: 101/500  loss: 0.03731985
epoch: 126/500  loss: 0.03796473
epoch: 151/500  loss: 0.02758416
epoch: 176/500  loss: 0.02679308
epoch: 201/500  loss: 0.02491123
epoch: 226/500  loss: 0.03305647
epoch: 251/500  loss: 0.02083281
epoch: 276/500  loss: 0.04825019
epoch: 301/500  loss: 0.04706486
epoch: 326/500  loss: 0.03616522
epoch: 351/500  loss: 0.03600014
epoch: 376/500  loss: 0.03080637
epoch: 401/500  loss: 0.03649246
epoch: 426/500  loss: 0.02524248
epoch: 451/500  loss: 0.03338098
epoch: 476/500  loss: 0.02643787
epoch: 500 loss: 0.03609372

Duration: 4 seconds


In [114]:
#Save Model

if len(losses) == epochs*(samplenum/batchsize):
    print('saving...')
    #torch.save(model.state_dict(), 'Trained_Model5000.pt')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

IndentationError: expected an indented block (<ipython-input-114-743b15137b18>, line 5)

In [ ]:
samplenum
